In [2]:
%load_ext autoreload
%autoreload 2

import torch
import tensorflow as tf
import torchvision
import numpy as np
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-06 15:35:05.041422: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 15:35:05.886062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-06 15:35:05.886295: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

### utils

In [3]:
def unpack_dataset(data_path):
    import glob
    import numpy as np
    from PIL import Image
    import tensorflow as tf
    import os
    import pandas as pd

    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except:
            pass

    if not os.path.exists('data'):
        os.mkdir('data')

    for split in ['train', 'val', 'test']:
        filenames = glob.glob(f'{data_path}/{split}/*.tfrec')
        if split in ['train', 'val']:
            tfrec_format = {
                "image": tf.io.FixedLenFeature([], tf.string),
                'class': tf.io.FixedLenFeature([], tf.int64),
                'id': tf.io.FixedLenFeature([], tf.string),
           }
        else:
            tfrec_format = {
                "image": tf.io.FixedLenFeature([], tf.string),
                'id': tf.io.FixedLenFeature([], tf.string),
           }

        dataset = tf.data.TFRecordDataset(filenames)
        parse_img_fn = lambda x: tf.io.parse_single_example(x, tfrec_format)
        dataset = dataset.map(parse_img_fn)

        idx = []
        classes = []
        filenames_jpg = []

        for sample in dataset.enumerate():
            id_val = sample[1]['id'].numpy().decode('utf-8')
            if split in ['train', 'val']:
                class_val = sample[1]['class'].numpy()
            else:
                class_val = None
            img = tf.image.decode_jpeg(
                sample[1]['image'], channels=3).numpy()
            img = Image.fromarray(img)
            filename = f'{id_val}.jpg'

            idx.append(id_val)
            classes.append(class_val)
            filenames_jpg.append(filename)

            img.save(f'data/{filename}')

        df = pd.DataFrame({'id': idx, 'filename':filenames_jpg, 'class': classes})
        df.to_csv(f'data/{split}.csv')


def get_data(url, path, filename):
    from urllib.request import urlretrieve
    import os

    # filename = f"{filename}.{url.split('.')[-1]}"
    
    if filename not in os.listdir(path):
        print('Downloading the dataset ...')
        urlretrieve(url, f'{path}/{filename}')
        print('Dataset downloaded.')
    else:
        print('Dataset is already present.')

### dataset

In [4]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import torch
from torchvision import transforms
from PIL import Image
import numpy as np

class Dataset_ptm(Dataset):
    def __init__(self, csv_path, data_path=None, transform=None, augment=False):
        if data_path is None:
            data_path = csv_path.split('/')
            data_path = '/'.join(data_path[:-1])
        self.data_path = data_path
        if transform is None:
            transform = self.get_transform()
        self.transform = transform
        self.augment = augment
        self.df = pd.read_csv(csv_path)

    def get_transform(self):
        # I wanted to calculate mean and std for entire dataset,
        # but this will do for now
        transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            ])
        return transform

    def get_img(self, idx=None):
        if idx is None:
            idx = np.random.randint(0, self.df.shape[0])
        img = Image.open(
            f'{self.data_path}/{self.df.iloc[idx]["filename"]}'
        ).convert('RGB')
        return(img)
    
    def augment_image(self, img):
        aug_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(15),
        ])
        return(aug_transform(img))
    
    def __getitem__(self, idx):
        df_idx = self.df.iloc[idx]
        img = self.get_img(idx)
        c = df_idx['class']
        img = self.transform(img)
        if self.augment:
            img = self.augment_image(img)
        return img, c

    def __len__(self):
        return self.df.shape[0]

### model

In [5]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import numpy as np
from time import time
from torcheval.metrics.functional import multiclass_f1_score

class VGG11(nn.Module):
    def __init__(self, conv_arch=None, resolution=(192, 192)):
        super().__init__()

        self.resolution = resolution
        self.pool_kernel_size = 2
        if conv_arch is None:
            pass
        self.layers = self.make_net(conv_arch)

        self.device = torch.device("cuda") \
            if torch.cuda.is_available() else torch.device("cpu")
        torch.device(self.device)
        self.to(self.device)

        self.loss_fn = nn.CrossEntropyLoss()
        self.score_fn = multiclass_f1_score

        learning_rate = 0.01
        weight_decay = 5e-4
        self.optimizer = optim.Adam(self.parameters(), weight_decay=weight_decay, lr=learning_rate)

    def vgg_block(self, num_convs, num_in, num_out):
        layers = []
        for ic in range(num_convs):
            layers.append(
                nn.Conv2d(in_channels=num_in, out_channels=num_out,
                    kernel_size=3, padding=1)
            )
            layers.append(nn.ReLU())
            num_in = num_out + 0
        layers.append(nn.MaxPool2d(kernel_size=self.pool_kernel_size, stride=2))
        return layers
    
    def make_net(self, conv_arch):
        layers = []
        resolution_after_conv = \
            np.array(self.resolution) // (self.pool_kernel_size**len(conv_arch))
        
        num_in = 3
        for ic, (num_convs, num_out) in enumerate(conv_arch):
            block = self.vgg_block(num_convs, num_in, num_out)
            layers += block
            num_in = num_out + 0

        fc_dim_in = resolution_after_conv.prod()*conv_arch[-1][-1]
        layers = nn.Sequential(
            *layers,
            nn.Flatten(),
            nn.Linear(fc_dim_in, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 104),
            nn.Softmax(dim=1))
        return nn.Sequential(*layers)
    
    def forward(self, img):
        out = img + 0.0
        for layer in self.layers:
            out = layer(out)
        return(out)

    def fit(self, dataset_train, dataset_val):
        print_period = 200
        # print_period = len(dataset_train) // 100
        num_epochs = 10
        batch_size = 16
        loader_train = DataLoader(dataset_train, shuffle=True, batch_size=batch_size)
        loader_val = DataLoader(dataset_val, shuffle=True, batch_size=batch_size)

        for epoch in range(num_epochs):
            s = f'Epoch {epoch+1} started.'
            print(s)
            t_epoch_start = time()
            losses_ep = []
            targets_true = []
            targets_pred = []

            for phase in ['train', 'val']:
                if phase == 'train':
                    self.train()
                    torch.set_grad_enabled(True)
                    loader = loader_train
                elif phase == 'val':
                    self.eval()
                    torch.set_grad_enabled(False)
                    loader = loader_val
                
                for i, (xb, yb) in enumerate(loader):
                    xb = xb.to(self.device)
                    yb = yb.to(self.device)
                    yb_pred = self(xb)
                    loss = self.loss_fn(yb_pred, yb)
                    losses_ep.append(loss.item())
                    targets_true.append(yb)
                    targets_pred.append( yb_pred.argmax(axis=1) )

                    if phase == 'train':
                        self.optimizer.zero_grad()
                        loss.backward()
                        self.optimizer.step()

                    if (i + 1) % print_period == 0 and phase == 'train':
                        s = f'Epoch: {epoch+1}, batch {i+1}. Train mean loss: {np.mean(losses_ep): 2.5f}'
                        print(s)
            t_epoch_end = time()
            dtime = round(t_epoch_end - t_epoch_start, 1)
            y_train = [y for (_, y) in dataset_train]
            y_val = [y for (_, y) in dataset_val]
            y_train_pred = [ self(x) for (x, _) in dataset_train ]
            y_val_pred = [ self(x) for (x, _) in dataset_val ]

            score_train = multiclass_f1_score(y_train_pred, y_train)
            score_val = multiclass_f1_score(y_val_pred, y_val)
            s = f'Epoch {epoch+1} train score: {score_train}, val score: {score_val}, time: {dtime}'
            print(s)

In [6]:
# from utils import get_data, unpack_dataset

url = r'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/21154/1243559/compressed/tfrecords-jpeg-192x192.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1678079006&Signature=q3WipKoyorkTnr%2F%2FpVkGYlEUJhvGDQDa6kHjAQsuDY51Q8X9yQm%2FVOZeLoRezxHGVKCubAePrvHkiFl%2BYNniJUhi46eJyZfi9DWh9dzfup9A%2F2EBBeru9DkP7VwMqW%2Bbyz8xJxjxL1utgOk%2FYLvtRmricDHZc7FTd7g6r7nUGE61854UFN6RqzYbrkLgS%2FQNnL6s8bNroFhL7UKaPhDh0RRWhymBHVGOGH8FRZoH%2F1ppIz7XhbTnD4qsffzcJ6upk8XmRc6TbX5MdF49W69DPlKZNdj8WbQP5YVHlGfQMG2406%2F%2Falb3FXVnFiVJUKiaPNdWtOW0D%2BJLucype21r9g%3D%3D&response-content-disposition=attachment%3B+filename%3Dtfrecords-jpeg-192x192.zip'
data_path_tf = 'data_tf'
data_filename = 'data.zip'
data_path = './data'

get_data(url, '.', data_filename)

from zipfile import ZipFile
with ZipFile(data_filename, 'r') as f:
    f.extractall(data_path_tf)

unpack_dataset(f'{data_path_tf}')

Dataset is already present.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-03-06 15:35:09.478354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-06 15:35:09.478396: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-06 15:35:09.478430: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (eg901-laptop): /proc/driver/nvidia/version does not exist
2023-03-06 15:35:09.478985: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-06 15:35:09.525838: I tensorflow/core/grappler/optimizers/data/

In [20]:
from dataset import Dataset_ptm
from torch.utils.data import DataLoader
dataset_train = Dataset_ptm('data/train.csv', augment=True)
dataset_val = Dataset_ptm('data/val.csv')
dataset_test = Dataset_ptm('data/test.csv')
loader_train = DataLoader(dataset_train, shuffle=True)
loader_val = DataLoader(dataset_val, shuffle=True)
loader_test = DataLoader(dataset_test, shuffle=True)

In [21]:
from model import VGG11

model = VGG11(
    conv_arch=(
        (1, 16),
        (1, 32),
        (2, 64),
        (2, 128),
        (2, 128)
    )
)

model.fit(dataset_train, dataset_val)


Epoch 1 started.
Epoch: 1, batch 200. Train mean loss:  4.61710


KeyboardInterrupt: 